In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import json
import pandas as pd
import numpy as np
import re
import plotly.offline as py
import plotly.graph_objects as go
import math
import pickle

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
file = "sim_matrix_final.json"
with open(path+file, 'r') as file_json:
    # Usa json.load() per caricare il contenuto del file JSON in un dizionario
    sim_matrix_final = json.load(file_json)

In [3]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
filename = "final_df.json"
df = pd.read_json(path+filename, lines=True)

In [4]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
file = "n_different_words_per_topic.json"
with open(path+file, 'r') as file_json:
    # Usa json.load() per caricare il contenuto del file JSON in un dizionario
    n_different_words_per_topic = json.load(file_json)

In [5]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI General\\TESI_FINAL_STAGE\\FASE 6\\DFs\\"
file = "n_words_per_topic.json"
with open(path+file, 'r') as file_json:
    # Usa json.load() per caricare il contenuto del file JSON in un dizionario
    n_words_per_topic = json.load(file_json)

In [6]:
topics = list(df['GPT_label'].unique())

In [7]:
G = nx.Graph()

In [8]:
for topic in topics:
    G.add_node(topic, size = math.log(n_words_per_topic[topic]))

In [9]:

for tx in topics:
    for ty in topics:
        if(sim_matrix_final[tx][ty] > 0.78 and tx != ty):
            G.add_edge(tx,ty)

In [10]:
pos_ = nx.circular_layout(G)

In [11]:
def make_edge(x, y, text):
    
    '''Creates a scatter trace for the edge between x's and y's with given width

    Parameters
    ----------
    x    : a tuple of the endpoints' x-coordinates in the form, tuple([x0, x1, None])
    
    y    : a tuple of the endpoints' y-coordinates in the form, tuple([y0, y1, None])
    
    width: the width of the line

    Returns
    -------
    An edge trace that goes between x0 and x1 with specified width.
    '''
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = 1,
                                   color = 'black'),
                       hoverinfo = 'none',
                       text      = ([text]),
                       mode      = 'lines')

In [12]:
edge_trace = []
for edge in G.edges():
    char_1 = edge[0]
    char_2 = edge[1]
    
    x0,y0 = pos_[char_1]
    x1,y1 = pos_[char_2]
    
    text = char_1 + '--' + ': ' + str(G.edges()[edge])
    
    trace = make_edge([x0,x1,None],[y0,y1,None],text)
    edge_trace.append(trace)

In [13]:
node_trace = go.Scatter(x=[],
                        y=[],
                        text=[],
                        textposition='top right',
                       textfont_size = 8,
                       mode = 'markers+text',
                       hoverinfo = 'text',
                       marker = dict(color = [],
                                    size = [],
                                    line = None))
for node in G.nodes():
    x,y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([(4* G.nodes()[node]['size'])] )
    node_trace['text'] += tuple(['<b>' + node + '</b>'])

In [14]:
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)'
)


fig = go.Figure(layout = layout)

for trace in edge_trace:
    fig.add_trace(trace)

fig.add_trace(node_trace)

fig.update_layout(showlegend = False, height = 1000, width = 1500,    plot_bgcolor='lightgray', paper_bgcolor='lavender')

fig.update_xaxes(showticklabels = False)

fig.update_yaxes(showticklabels = False)

fig.show()

In [19]:
file = 'rete.pickle'
with open(path+file, 'wb') as f:
    pickle.dump(G, f)

In [34]:
G.number_of_edges()

185